<a href="https://colab.research.google.com/github/ajdema/neural_style_transfer/blob/main/painting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable

from PIL import Image
import scipy.misc

import torch.utils.data
import torchvision.datasets as datasets
import imageio
import numpy as np

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Computes the Gram matrix of a given tensor essentially every column is multiplied with every row in the matrix, we can think of the spatial information that was contained in the original representations to have been “distributed”. The Gram matrix instead contains non-localized information about the image, such as texture, shapes, and weights — style!

In [ ]:
class GramMatrix(nn.Module):

    def forward(self, input):
        a, b, c, d = input.size()
        features = input.view(a * b, c * d)
        G = torch.mm(features, features.t())

        return G.div(a * b * c * d)

## StyleCNN Initialization

In this section of code, we initialize the StyleCNN object, which is designed for neural style transfer. The key variables and components include:

- **Input Images:**
  - `style`: Style image for the transfer.
  - `content`: Content image to be stylized.
  - `pastiche`: Parameterized representation of the stylized image.

- **Loss Computation Parameters:**
  - `content_layers`: Layers used for computing content loss.
  - `style_layers`: Layers used for computing style loss.
  - `content_weight`: Weight multiplier for content loss.
  
  - `style_weight`: Weight multiplier for style loss.

- **Pretrained Network:**
  - `loss_network`: VGG-19 pretrained network used to obtain intermediate representations.

- **Gram Matrix Computation:**
  - `gram`: GramMatrix module for computing Gram matrices.

- **Loss Computation:**
  - `loss`: Mean Squared Error (MSE) loss function for content and style losses.

- **Optimizer:**
  - `optimizer`: LBFGS optimizer used for optimizing the pastiche image.

- **CUDA Availability:**
  - `use_cuda`: Boolean indicating whether CUDA (GPU acceleration) is available. If true, moves relevant components to GPU.




In [ ]:
class StyleCNN(object):
    def __init__(self, style, content, pastiche):
        super(StyleCNN, self).__init__()

        self.style = style
        self.content = content
        self.pastiche = nn.Parameter(pastiche.data)

        self.content_layers = ['conv_4']
        self.style_layers = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']
        self.content_weight = 1
        self.style_weight = 1

        self.loss_network = models.vgg19(pretrained=True)

        self.gram = GramMatrix()
        self.loss = nn.MSELoss()
        self.optimizer = optim.LBFGS([self.pastiche])

        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.loss_network.cuda()
            self.gram.cuda()
    def train(self):
      def closure():
          self.optimizer.zero_grad()

          pastiche = self.pastiche.clone()
          pastiche.data.clamp_(0, 1)
          content = self.content.clone()
          style = self.style.clone()

          content_loss = 0
          style_loss = 0

          i = 1
          not_inplace = lambda layer: nn.ReLU(inplace=False) if isinstance(layer, nn.ReLU) else layer
          for layer in list(self.loss_network.features):
              layer = not_inplace(layer)
              if self.use_cuda:
                  layer.cuda()

              pastiche, content, style = layer.forward(pastiche), layer.forward(content), layer.forward(style)

              if isinstance(layer, nn.Conv2d):
                  name = "conv_" + str(i)

                  if name in self.content_layers:
                      content_loss += self.loss(pastiche * self.content_weight, content.detach() * self.content_weight)

                  if name in self.style_layers:
                      pastiche_g, style_g = self.gram.forward(pastiche), self.gram.forward(style)
                      style_loss += self.loss(pastiche_g * self.style_weight, style_g.detach() * self.style_weight)

              if isinstance(layer, nn.ReLU):
                  i += 1

          total_loss = content_loss + style_loss
          total_loss.backward()

          return total_loss

      self.optimizer.step(closure)
      return self.pastiche

Some utility functions for loading and saving images.

In [ ]:
imsize = 256

loader = transforms.Compose([
             transforms.Resize(imsize),
             transforms.ToTensor()
         ])

unloader = transforms.ToPILImage()

def image_loader(image_name):
    image = Image.open(image_name)
    image = Variable(loader(image))
    image = image.unsqueeze(0)
    return image

def save_image(input, path):
    image = input.data.clone().cpu()

    size = (image.size(1), image.size(2), image.size(3))

    image = image.view(*size)

    image = image.numpy()
    image = (image * 255).astype(np.uint8)  # Convert to uint8

    image = np.clip(image, 0, 255)

    imageio.imsave(path, image.transpose(1, 2, 0))

In [ ]:
# CUDA Configurations
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

# Content and style
path = "/content/drive/MyDrive/painting_completion/"
style = image_loader(path + "Mona_Lisa.jpg").type(dtype)
content = image_loader(path + "Leonardo_da_Vinci_Adoration_of_the_Magi.jpg").type(dtype)

pastiche = image_loader(path + "Leonardo_da_Vinci_Adoration_of_the_Magi.jpg").type(dtype)
pastiche.data = torch.randn(pastiche.data.size()).type(dtype)

num_epochs = 401

def main():
    global pastiche
    style_cnn = StyleCNN(style, content, pastiche)

    for i in range(num_epochs):
        pastiche = style_cnn.train()

        if i % 10 == 0:
            print("Iteration: %d" % (i))

            path = "/content/drive/MyDrive/painting_completion/outputs/%d.png" % i
            pastiche.data.clamp_(0, 1)
            save_image(pastiche, path)

main()

Iteration: 0
Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100
Iteration: 110
Iteration: 120
Iteration: 130
Iteration: 140
Iteration: 150
Iteration: 160
Iteration: 170
Iteration: 180
Iteration: 190
Iteration: 200
Iteration: 210
Iteration: 220
Iteration: 230
Iteration: 240
Iteration: 250
Iteration: 260
Iteration: 270
Iteration: 280
Iteration: 290
Iteration: 300
Iteration: 310
Iteration: 320
Iteration: 330
Iteration: 340
Iteration: 350
Iteration: 360
Iteration: 370
Iteration: 380
Iteration: 390
Iteration: 400
